In [1]:
#hide
from fastbook import *

In the previous chapters we learn about the CNN in detailed and also about the resent architectures 
which we were using in all our computer vision models.In this chapter we would be learning about the 
state of art architectures we used for tabular,NLP and Image segmentation modules..

# Application Architectures Deep Dive

Now we are in such condition that we can understand the state of art architectures for computer vision
,NLP and tabular modules.We would thus be learning about all the details of the different fastai 
applications.

We will also go through the DataPreprocessing pipeline we had used for Images in Siamese Network model
and learn how we can build the pretrained models for new applications..

We would start with Computer Vision first..

## Computer Vision

We had built Image Classifiers and Image Segmentation for that we used cnn_learner and unet_learner 
for the models.We would learn about creating the learner objects we used directly in Rest of the 
chapters in this book.We would learn about what exactly happens when cnn_learner is called and we pass
the arguments..

### cnn_learner

We pass architectures through these functions which make the body of the network.Most of the times we
passed ResNet as the architecture about which we learnt in the previous chapter that it uses skip
connections and identical mapping.As these are pretrained models so weights are downloaded and loaded
in ResNet.

Then we learnt about transfer learning where for some layers we use pretrained weights and the later
layers are discarded and trained again.So the network has to be cut.We discard the final layer from 
the network which is responsible for specific classification.Everything after adaptive average pooling
layer is sliced off.To search from where the layer has to be cut off,we use a dictionary which shows
the contents of the model architecture from its body,head everything.It is called "__model_meta__" and
and we pass the architecture through it.Let's see what is the output we get..

Head of the model is part of the model which is used for a particular task.In a CNN,that is adaptive
average pooling layer.
Stem is the initial layers and the body includes stem also.

In [5]:
#Seeing the model dictionary for resnet-50
model_meta[resnet50]

{'cut': -2,
 'split': <function fastai.vision.learner._resnet_split(m)>,
 'stats': ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])}

"__Cut__" gives the point after which the layers are cut off.All the layers before this point are kept
for transfer learning as they are pretrained weights.It is basically cutting off the head.We thus put
a new head in our model.We create it using the create_head function.

In [6]:
create_head(20,2)

Sequential(
  (0): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=1)
    (mp): AdaptiveMaxPool2d(output_size=1)
  )
  (1): Flatten(full=False)
  (2): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.25, inplace=False)
  (4): Linear(in_features=20, out_features=512, bias=False)
  (5): ReLU(inplace=True)
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.5, inplace=False)
  (8): Linear(in_features=512, out_features=2, bias=False)
)

Using this function we add how many layers we want to add to our model at the end.We can also choose 
the number of dropouts and the pooling layers to use.Fastai by default adds average pooling and max 
pooling together and combines them in AdaptiveConcatPool2d layer.It's not done commonly and generally
other libraries add one layer at a time.

Fastai by default adds two linear layers and not one in the CNN head part(the final part of the layer)
.It is because transfer learning can then be used for applications in diffrent domains.But if a single
layer is added,it cannot be used.Two linear layers allow transfer learning to be quick and easy.

Let's understand the unet_learner we used for segmentation previously...

### unet_learner

Image segmentation is also a problem belonging to Computer Vision.It is relatively challenging then
Classification because every single pixel is labelled and that's why  predictions are like pixel grid.
Some other problems such as increasing resolution of the image,converting black and white image to 
colored(colorization) are also like Segmentation.In all these problems we have an Image as the input
which is converted into other image with same dimension but the pixels are labelled.These are also
known as "__generative vision models__".

The process of modifying the model architecture is same here also.We first cut off the final layers 
and create a CNN head.Starting with ResNet the adaptive pooling layer and everything after that is 
removed and replaced with the head we create.

CNN classifies the image.We start with a 224 px image in the starting but at the end of the network
after we have cut off the head we have a 7 X 7 grid containing convolutional activations.How do we 
convert it into an Image of 224 px.

This is done through deep learning only meaning we use neural networks to build the segmentation mask.
At the end of ResNet body we have a 7 X 7 grid so we need to apply some layer which can increase the 
grid size.One of the easy ways to do this is we replace every pixel in the 7 X 7 grid with 4 pixels
in a 2 X 2 grid.These pixels would have same value and this replacement is known as nearest neighbour
interpolation.PyTorch has a custom layer for this and we can use that.So first we create a head 
containing a stride 1 convolutional layer along with 2 nearest neighbor interpolation layers.The 
results obtained are not that great though.

# Transposed convolution

Transposed convolution is also one of the apporaches used.Instead of the convolutional and the nearest
neighbor layer we add a half stride convolutional layer.It is Similar to regular convolution but first
a zero padding is added between the pixels.Below figure explains that how transposed convolution is 
applied:-    
<img alt="A transposed convolution" width="815" caption="A transposed convolution (courtesy of Vincent Dumoulin and Francesco Visin)" id="transp_conv" src="images/att_00051.png">

The above figure explains that how zero padding is inserted(the grey grids).This results in an 
increased input size.Transposed convolution can be applied using the fastai's Conv layer and passing 
the transpose as True which creates transposed convolution.

These approaches do not work very well but.As 7 X 7 grid cannot be used to create a 224 X 224 px image
as output.There is not enough information which can be represented.Thus skip connections are used 
which form the basis of the ResNet but instead of skipping convolution layers,the part from the
activations in the ResNet body to the activations in the transposed convolution are skipped.This 
approach is described in a research paper which describes UNet networks used for biomedical image 
segmentation.Though initially it was developed for biomedical applications,but it can be used for 
many generative vision tasks such as colorization,super resolutiona and segmentation.
The below figure explains the unet architecture in much detailed..
<img alt="The U-Net architecture" width="630" caption="The U-Net architecture (courtesy of Olaf Ronneberger, Philipp Fischer, and Thomas Brox)" id="unet" src="images/att_00052.png">

As we mentioned before the activations from the ResNet body are skipped till the transposed 
convolutions on the opposite side.The CNN body is present on the left side.It's a regular CNN and not
ResNet and it has a 2 X 2 max pooling layer and not stride 2 convolutions.The transposed convolutional
layers are on the opposite side.("up-conv").The skip connections are the ones shown in gray arrows 
from left to right.Since the shape of the network becomes like this that's why it is called 
"U-Net" architecture.

Using this architecture the input is not only the 7 X 7 grid in the previous layer but the higher 
resolution grid in the ResNet head also.This enables Unet to use all the information in the original
image which was not possible earlier.

Let's write a custom model architecture for SiaMese network which we had also done earlie..

### A Siamese Network

In a Siamese network there are pairs of images labelled as True or False as input.It tells if both the
images belong to the same class or not.Below is a detailed code starting from downloading the data to
labelling it and creating the DataLoaders object for training.

This is the same code we had gone through in the Chapter-11 while discussing midlevel API.It contains
a class SiameseImage which subclasses it to contain two images and a boolean (True or False) if the 
image belongs to the same class or not.A show method is implemented to display the images concatenated
into a single one separated by a black line.An if condition is put that if the images are not tensors.
Then they are converted into tensors of same size.A zero matrix is created for the separating black 
line and this class returns concatenated image tensor and the line.

In [1]:
#importing all the modules from fastai.vision
from fastai.vision.all import *
path = untar_data(URLs.PETS)#downloading and extracting the pets dataset
files = get_image_files(path/"images")#getting the image files

class SiameseImage(fastuple): #SiameseImage declaration
    def show(self, ctx=None, **kwargs): 
        img1,img2,same_breed = self #the three parameters passed
        #(image1,image2,the boolean if they are of same breed oe not)
        if not isinstance(img1, Tensor): #conditions if images are not tensors
            if img2.size != img1.size: #test if the images are not of same size
                img2 = img2.resize(img1.size)#resize the image if they are not equal size
            t1,t2 = tensor(img1),tensor(img2)#convert the images into tensors
            t1,t2 = t1.permute(2,0,1),t2.permute(2,0,1)#change the dimension of the image tensors 
            #using permute
            #else if images are tensors
        else: t1,t2 = img1,img2
        line = t1.new_zeros(t1.shape[0], t1.shape[1], 10)#a zero tensor for the black line.The shape 
        #is similar to the Image tensors
        return show_image(torch.cat([t1,line,t2], dim=2), #concatenate the two images and the line.
                          title=same_breed, ctx=ctx)
        #display the concatenated images using the show_image
    
#Label function for naming the images.Using Regex extracts the breed name from the filename
def label_func(fname):
    return re.match(r'^(.*)_\d+.jpg$', fname.name).groups()[0]

class SiameseTransform(Transform):
    def __init__(self, files, label_func, splits):
        self.labels = files.map(label_func).unique()
        self.lbl2files = {l: L(f for f in files if label_func(f) == l) for l in self.labels}
        self.label_func = label_func
        self.valid = {f: self._draw(f) for f in files[splits[1]]}
        
    def encodes(self, f):
        f2,t = self.valid.get(f, self._draw(f))
        img1,img2 = PILImage.create(f),PILImage.create(f2)
        return SiameseImage(img1, img2, t)
    
    def _draw(self, f):
        same = random.random() < 0.5
        cls = self.label_func(f)
        if not same: cls = random.choice(L(l for l in self.labels if l != cls)) 
        return random.choice(self.lbl2files[cls]),same
    
splits = RandomSplitter()(files)#training and validation split
tfm = SiameseTransform(files, label_func, splits)#labelling the images
tls = TfmdLists(files, tfm, splits=splits)#get the transformed lists of the images
dls = tls.dataloaders(after_item=[Resize(224), ToTensor], #creating dataloaders
    after_batch=[IntToFloatTensor, Normalize.from_stats(*imagenet_stats)],num_workers=0,bs=32)

We can now build a custom model and train thid model using a pretrained architecture.It takes two 
images as input and concatenates the result and passes them through a head part which returns two 
predictions.We would create head and body of the model ahead.

In [2]:
#Creating custom model for SiameseModel
class SiameseModel(Module):
    def __init__(self, encoder, head):
        self.encoder,self.head = encoder,head#encoder and the head part of the model
    
    def forward(self, x1, x2):#pass the two images
        ftrs = torch.cat([self.encoder(x1), self.encoder(x2)], dim=1)#pass them through encoder and 
        #concatenate them
        return self.head(ftrs)#pass them through the head part 

We need to define encoder now for encoding the image.It is done using the resnet architecture.That is
done using the function create_body through which we pass the resnet34 architecture and cut=-2 so as 
to cut the last two layers which will be our custom layers we would be defining again.

In [3]:
#Creating encoder(body) of the model
encoder = create_body(resnet34, cut=-2)

In [11]:
encoder

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

encoder's last layer has 512 features.So the last layers or the head part would have input doubled 
because we have two images and it would be doubled again because of the concat pooling layer which 
adds two layers by default.So the head part of the model is created using the create_head function 
through which we pass the total no of features:512 X 4.

In [13]:
create_head??

In [4]:
#Creating head part
head = create_head(512*4, 2, ps=0.5)

Next we pass the encoder and head through the SiameseModel class to get the final model architecture.

In [5]:
#passing encoder,head through the SiameseModel class.
model = SiameseModel(encoder, head)

Now we can create learner.But before that we have to define the loss function.We use CrossEntropyLoss
() only but we create a loss_func separately because the targets are in booleans and not integers.So
They are first converted into integers and then returned from the loss_func.We pass the targets and 
predictions through the loss_func.

In [6]:
#defining loss function
def loss_func(out, targ):
    return nn.CrossEntropyLoss()(out, targ.long())#returns cross entropy loss betweeen the predictions
#and the targets converted into integers.

We also define a splitter function.It tells fastai library to split the model into parameter groups.
They are used to train the head of the model.Two different parameter groups are needed for the encoder
and the head."__params__" function gives all parameters of the passed architecture.Thus we pass the 
encoder and head differently through paramas to return the separate parameters for both.In the below 
code cell the function "__siamese_splitter__" is defined which returns these separate parameters for
head and encoder.

In [7]:
#splitter function for splitting the parameters
def siamese_splitter(model):
    return [params(model.encoder), params(model.head)]
#returns the parameter sets for encoder and head

Now as we have defined loss function and splitter we can define the learner by passing the dataloaders,the model architecture,the loss function,the splitter and the metrics used.

We aren't directly using the cnn_learner so the previous layers prior to head are frozen using learn.
freeze().Then we train the last head of the model in the usual way by training for some epochs and a
learning rate of 3 X 10^-3.

In [8]:
#Creating the learner 
learn = Learner(dls, model, loss_func=loss_func, 
                splitter=siamese_splitter, metrics=accuracy)
learn.freeze()#freezing the previous layers of the model

In [9]:
#Training the last layer only for 4 epochs.
learn.fit_one_cycle(4, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.540838,0.395717,0.828823,02:05
1,0.424815,0.346322,0.841001,01:56
2,0.328580,0.213767,0.921516,01:57
3,0.296726,0.183778,0.924222,01:57


We unfreeze the previous layers and train the whole model then with discriminative learning rates that
is increasing learning rate with increasing layers.lower for the body but higher for head.We thus
train the model by passing 4 epochs and a range of learning rates.

In [14]:
#unfreezing the layers
learn.unfreeze()
#Training the model for 4 epochs and at discriminative learning rates
learn.fit_one_cycle(4, slice(1e-6,1e-4))

epoch,train_loss,valid_loss,accuracy,time
0,0.270499,0.176588,0.928958,02:17
1,0.273814,0.173865,0.927605,02:18
2,0.266506,0.177017,0.928281,02:17
3,0.257422,0.173762,0.927605,02:17


92.7% is a good accuracy for such model.We learnt about creating learners for computer vision models.
We can now learn about NLP learners in the same way.

## Natural Language Processing

We had trained an AWD-LSTM language model from scratch in the previous chapters.We can define custom 
models and turn it into a transfer learning classifier.It can be done in the same way like we did for
cnn_learner.We use stacked RNN as the body of the architecture.The encoder provides an activation 
for every input word.The Language model in general predicts every next word.

We use the same approach we had used in the previous chapter for NLP_dive.It uses the one from the 
ULMFiT paper which is based on the BPTT.(Back Propogation Through time) which means calculating 
gradients for some past time steps only instead of the whole stream.It detaches the gradient history
at every step.

We use a for loop in the classifier which iterates over every batch in the sequence.The activation for
every batch is stored.At the end they are averaged and max pooled.

Though the data is separated into mini-batches but each word is considered as one token as every word
has a separate label.The texts are not necessarily of same length though,therefore they cannot be put 
in the same array.

Like in CNN we use padding for images to add some extra pixels we can do the same with text to add
some extra words.From the text we identify the text with greatest length and then fill the text with
tokens "xxpad".In this way texts are made of same size.If there is a text such that it has batches of 
2000 tokens as well as 10 tokens then a lot of padding tokens have to be added which leads to wastage 
of memory.So,texts of comparable sizes are put together.Fastai does this automatically while creating
dataloaders.

Next we would discuss about fastai tabular models.We won't be talking about collaborative filtering
models separately as they are similar to tabular models.

## Tabular

We won't be talking much about the whole architecture of the TabularModel .We would be talking more
about the forward function.
Let's see what all is included in the forward method for TabularModel:-
The code looks like:-
    
    ```python
if self.n_emb != 0:
    x = [e(x_cat[:,i]) for i,e in enumerate(self.embeds)]
    x = torch.cat(x, 1)
    x = self.emb_drop(x)
if self.n_cont != 0:
    x_cont = self.bn_cont(x_cont)
    x = torch.cat([x, x_cont], 1) if self.n_emb != 0 else x_cont
return self.layers(x)
```

Let us now understand every line of the above code.

"__self.n_emb!=0__" tests if there are any embeddings in the data.If there are embedding matrices then
we can get the activations through "__x = [e(x_cat[:,i]) for i,e in enumerate(self.embeds)]__".They 
are later combined into a tensor through "__x = torch.cat(x, 1)__".Next we apply dropout using the 
"emb_drop"."__x = self.emb_drop(x)__".
Next we check if there are any continuous variables in the data using "__if self.n_cont != 0:__".They
are passed through the batch_norm layer using "__ x_cont = self.bn_cont(x_cont)__".They are combined
with the embedded activations."__x = torch.cat([x, x_cont], 1)__".At the end it is passed through 
three linear layers.And the activations are returned.


We have so far discussed about the basics of all the architectures for Image segmentation,
classification,NLP and tabular modules in the fastai.

## Wrapping Up Architectures

Whenever we want to understand the beneath details of any architecture,the source code in fastai is of
help.It is important to understand the meaning of the code.

We have almost investigated about all the steps of building a practical deep learning model now.In 
case of unlimited data and time we can train a big model for analysis which would then take a large 
time also to run.However most of the times we have memory and time constraints.That time its better to
train a small model.Initially we should train our model till the point that its starts getting overfit
and then we should apply some techniques to reduce that.

Initial steps include train your model till it gets overfit.Now to reduce overfitting below figure 
explains a broader view of the steps to be followed:-
    
<img alt="Steps to reducing overfitting" width="400" caption="Steps to reducing overfitting" id="reduce_overfit" src="images/att_00047.png">

Most of the people prefer using small models or regularization.But that is wrong.Infact using smaller 
model would be the least preferred step among all.If at all after trying all methods we are not able
to reduce overfitting then we can try small models.

The first step after overfitting should be to crate more data.It may include adding more labels or 
adding synthetic data by trying different data augmentation transforms.Mixup is one of the approaches
used though there are many available for all kinds of data.

After creating enough data,we should follow proper augmentation process.If that also is not reducing
overfitting,batch normalization layer can be added.

After following these steps if the model still overfits then we can apply regularization.Adding a
dropout to the last layer regularizes the model.Adding dropouts of different types helps in making the
model more flexible.Thus a large model with more regularization is more flexible and accurate than a 
small model with less regularization.

If any of these options don't work then only we try small architecture.

## Questionnaire

1. What is the "head" of a neural net?
1. What is the "body" of a neural net?
1. What is "cutting" a neural net? Why do we need to do this for transfer learning?
1. What is `model_meta`? Try printing it to see what's inside.
1. Read the source code for `create_head` and make sure you understand what each line does.
1. Look at the output of `create_head` and make sure you understand why each layer is there, and how the `create_head` source created it.
1. Figure out how to change the dropout, layer size, and number of layers created by `cnn_learner`, and see if you can find values that result in better accuracy from the pet recognizer.
1. What does `AdaptiveConcatPool2d` do?
1. What is "nearest neighbor interpolation"? How can it be used to upsample convolutional activations?
1. What is a "transposed convolution"? What is another name for it?
1. Create a conv layer with `transpose=True` and apply it to an image. Check the output shape.
1. Draw the U-Net architecture.
1. What is "BPTT for Text Classification" (BPT3C)?
1. How do we handle different length sequences in BPT3C?
1. Try to run each line of `TabularModel.forward` separately, one line per cell, in a notebook, and look at the input and output shapes at each step.
1. How is `self.layers` defined in `TabularModel`?
1. What are the five steps for preventing over-fitting?
1. Why don't we reduce architecture complexity before trying other approaches to preventing overfitting?

### Further Research

1. Write your own custom head and try training the pet recognizer with it. See if you can get a better result than fastai's default.
1. Try switching between `AdaptiveConcatPool2d` and `AdaptiveAvgPool2d` in a CNN head and see what difference it makes.
1. Write your own custom splitter to create a separate parameter group for every ResNet block, and a separate group for the stem. Try training with it, and see if it improves the pet recognizer.
1. Read the online chapter about generative image models, and create your own colorizer, super-resolution model, or style transfer model.
1. Create a custom head using nearest neighbor interpolation and use it to do segmentation on CamVid.

# Answers:-

Ans-1 Head of a neural net is that part of the network which is responsible for doing a specific task
in the network.Mostly these are the last layers of the network always.

Ans-2 Body of the network refers to everything in the model except the head part.It includes initial 
layers and the encoding part of the model.Body ends where the head starts.

Ans-3 Cutting a neural network means slicing off a part of neural network.Generally the final layers
are discarded off.In transfer learning it is important because rest part of the model contains 
pretrained layers.They don't need more training whereas the head of the model does specific task for
specific problem,so it has to be trained all over again.That's why the pretrained weights in the last
layers are discarded.

Ans-4 model_meta gives the dictionary of information we require to create a custom model from 
pretrained architecture.We just need to pass the architecture name through it.

Ans-8 AdaptiveconcatPool2d is a layer provided by fastai which by default applies average pooling and
max pooling together and concatenates its results.

Ans-9 If we have an activation grid of size 7 X 7 then we replace each pixel by 4 pixels in a grid 
size of 2 X 2.The values of all the four pixels in the square would be same and this is known as 
nearest neighbor interpolation.This allows us to increase the grid size in the CNN.

Ans-10 To upsample the grid size in CNN,we use transposed convolution.We do a regular convolution here
only difference is that we first insert a zero padding.To get a transposed convolution,we have to pass
the parameters Transpose=True in the fastai's ConvLayer in the head we would create for the model.
Also known as stride-half convolution.

Ans-13 BPTT(BackPropogation through time) is used for updating the weights when RNN is used for some
sequential data like time series.The gradients are updated at each time step to save memory and time.

Ans-14 To handle different length texts in batches,we do padding by filling the texts which have short
length with special tokens called "xxpad".

Ans-16 In "__init__".

Ans-17 Collect more data,Data Augmentation,Adding generalized layers,Regularization,reduce model
complexity by trying small models.

Ans-18 Reducing model complexity in initial steps would reduce the ability of the model to learn 
important relationships.